In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna import Trial, visualization


In [2]:
train_df =  pd.read_csv('sub_train.csv')
test_df  =  pd.read_csv('sub_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

startdate
climateregions__climateregion


In [4]:
exclude_cols = ['index', 'startdate']
target=["contest-tmp2m-14d__tmp2m"]
main_attrs = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(main_attrs))

Main features: 59


In [6]:
main_attrs

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [10]:
X = train_df[main_attrs].values
y = train_df[target[0]].values
Xtest = test_df[main_attrs].values

X.shape, y.shape, Xtest.shape

((375734, 59), (375734,), (31354, 59))

In [140]:
class StackingCVRegressorRetrained(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, regressors, meta_regressor, n_folds=2, use_features_in_secondary=False):
        self.regressors = regressors
        self.meta_regressor = meta_regressor
        self.n_folds = n_folds
        self.use_features_in_secondary = use_features_in_secondary
        self.regr_ = [clone(x) for x in self.regressors]
        self.meta_regr_ = clone(self.meta_regressor)
    def fit(self, train, y):
        if type(train) == pd.core.frame.DataFrame:            
            X = copy.deepcopy(train).values
            
            
        elif type(train) == np.ndarray:            
            X = copy.deepcopy(train)

        kfold = KFold(n_splits=self.n_folds, shuffle=True)

        out_of_fold_predictions = np.zeros((X.shape[0], len(self.regressors)))
        
        for i, regr in tqdm(enumerate(self.regr_)):
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = clone(regr)
                if i == 1:
                    instance.fit(X[train_idx], y[train_idx], eval_set=[(X[holdout_idx], y[holdout_idx])], verbose=True, early_stopping_rounds=100)
                else:
                    instance.fit(X[train_idx], y[train_idx])
                    
                out_of_fold_predictions[holdout_idx, i] = instance.predict(X[holdout_idx])

        
        all_predictions = np.zeros((X.shape[0], len(self.regressors)))
        for i, regr in enumerate(self.regr_):
            regr.fit(X, y)
            all_predictions[:, i] = regr.predict(X)

        
        
            X = train.values
        

        if self.use_features_in_secondary:
            self.meta_regr_.fit(np.hstack((X, out_of_fold_predictions)), y)
        else:
            self.meta_regr_.fit(out_of_fold_predictions, y)

        return self
    
    def predict(self, test):
        if type(test) == pd.core.frame.DataFrame:
            X = copy.deepcopy(test).values
        elif type(test) == np.ndarray:            
            X = copy.deepcopy(test)
        
        meta_features = np.column_stack([
            regr.predict(X) for regr in self.regr_
        ])
        
        if type(test) == pd.core.frame.DataFrame:      
            X = test.values

        if self.use_features_in_secondary:
            return self.meta_regr_.predict(np.hstack((X, meta_features)))
        else:
            return self.meta_regr_.predict(meta_features)

In [145]:
cat = CatBoostRegressor(iterations=5000, verbose=200)
gbr = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=350, verbose=1, min_samples_split=25, min_samples_leaf=25)
stack_model = StackingCVRegressorRetrained((gbr, cat), cat, use_features_in_secondary=True)

In [ ]:
stack_model.fit(X, y)

0it [00:00, ?it/s]

      Iter       Train Loss   Remaining Time 
         1          81.8211           15.71m
         2          69.0710           16.21m
         3          58.6294           16.62m
         4          49.9253           16.89m
         5          42.9270           16.75m
         6          36.8817           16.75m
         7          31.9056           16.73m
         8          27.8288           16.66m
         9          24.3049           16.57m
        10          21.4272           16.50m
        20           8.6286           16.01m
        30           5.4499           15.55m
        40           4.3224           14.94m
        50           3.7317           14.29m
        60           3.3621           13.73m
        70           3.1245           13.34m
        80           2.9320           12.80m
        90           2.7641           12.32m
       100           2.6156           11.86m
       200           1.8670            7.25m
